In [2]:
import json
import pandas as pd


items_path = "mv_items.json"
prices_path = "only_prices.json"
output_path = "mvideo_features.csv"

items = []
with open(items_path, "r", encoding="utf-8") as f:
    for line in f:
        items.append(json.loads(line))

with open(prices_path, "r", encoding="utf-8") as f:
    prices_list = json.load(f)
price_map = {p["productId"]: p for p in prices_list}

all_feature_names = set()
for item in items:
    for block in item["all_properties"]:
        for prop in block["properties"]:
            name = prop["name"]
            measure = prop.get("measure")
            if measure:
                name = f"{name} ({measure})"
            all_feature_names.add(name)
data_rows = []
for item in items:
    pid = item.get("product_id")
    row = {
        "product_id": pid,
        "name": item.get("name"),
        "brand": item.get("brand")
    }
    for block in item["all_properties"]:
        for prop in block["properties"]:
            name = prop["name"]
            value = prop.get("value")
            measure = prop.get("measure")
            if measure:
                name = f"{name} ({measure})"
            row[name] = value
    price_data = price_map.get(pid)
    if price_data:
        row["basePrice"] = price_data.get("basePrice")
        row["salePrice"] = price_data.get("salePrice")

    data_rows.append(row)

df = pd.DataFrame(data_rows)
df.to_csv(output_path, index=False, encoding="utf-8")


In [3]:
import pandas as pd

df = pd.read_csv("mvideo_features.csv")

print("📋 Все колонки:")
for col in df.columns:
    print(col)

print("\n📉 Количество пропущенных значений (NaN) по каждой колонке:")
nan_counts = df.isna().sum()
print(nan_counts[nan_counts > 0].sort_values(ascending=False))  


📋 Все колонки:
product_id
name
brand
Гарантия
Гарантия предоставляется
Страна
Год релиза
Серия
Состояние
Внешний вид
Разрешение экрана
Экран
Технология экрана
Тип экрана
Частота обновления (Гц)
Яркость (кд/кв.м)
Тип процессора
Количество ядер
Графический ускоритель
Операционная система
Встроенная память (ROM) (ГБ)
Количество основных камер (шт)
Основная камера МПикс
Съемка видео в портретном режиме
Цифровой зум (x)
Разрешение видеосъемки
Оптическая стабилизация
Количество фронтальных камер (шт)
Фронтальная камера МПикс
Поддержка SIM карт
Поддержка стандартов
Поддержка Wi-Fi
Технология NFC
Встроенный модуль Bluetooth
Поддержка Apple Pay
Встроенный барометр
Датчик ориентации экрана
Сенсор распознавания лица
Порт USB
Поддержка быстрой зарядки
Поддержка беспроводной зарядки
Стандарты быстрой зарядки
Датчик ускорения (G-sensor)
Разъем для наушников
Материал корпуса
Степень защиты
Поддержка GPS
Поддержка ГЛОНАСС
В комплекте
Поддержка MagSafe
Тип аккумулятора
Цвет
Цвет производителя
Габаритны

In [ ]:
import matplotlib.pyplot as plt

nan_df = nan_counts[nan_counts >= 0].sort_values(ascending=False).to_frame(name='nan_count')

plt.figure(figsize=(10, max(4, len(nan_df) * 0.3)))
nan_df['nan_count'].plot(kind='barh')
plt.xlabel('Количество NaN')
plt.ylabel('Колонка')
plt.title('Количество пропущенных значений по колонкам')
plt.tight_layout()
plt.savefig('nan_counts.png')
plt.close()